In [32]:
%load_ext autoreload
%autoreload 2
from notebook import *
# if get something about NUMEXPR_MAX_THREADS being set incorrectly, don't worry.  It's not a problem.

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Code Example: Linked list v.s. array

In [33]:
compare([do_render_code("linked_list/linked_list.c", show="depth_of_array"),do_render_code("linked_list/linked_list.c", show="depth_of_list")])

In [36]:
! make -C linked_list clean ; make -C linked_list
! echo "size,list,IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branches,branch_misses" > stats.csv
! echo -n "4096,0," >> stats.csv
! linked_list/linked_list 4096 0
! echo -n "4096,1," >> stats.csv
! linked_list/linked_list 4096 1

make: Entering directory '/nfshome/htseng/courses/CS203/demo/OoO_programming/linked_list'
rm -f linked_list linked_list.s
make: Leaving directory '/nfshome/htseng/courses/CS203/demo/OoO_programming/linked_list'
make: Entering directory '/nfshome/htseng/courses/CS203/demo/OoO_programming/linked_list'
cc -O3 -Wno-implicit-function-declaration -Wno-format-zero-length -DHAVE_LINUX_PERF_EVENT_H perfstats.c  linked_list.c -o linked_list
cc -S -O3 -Wno-implicit-function-declaration -Wno-format-zero-length -DHAVE_LINUX_PERF_EVENT_H linked_list.c 
make: Leaving directory '/nfshome/htseng/courses/CS203/demo/OoO_programming/linked_list'
depth: 4096
depth: 4096


In [37]:
display_df_mono(render_csv("stats.csv"))

,size,list,IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branches,branch_misses
0,4096,0,29815,22783,0.764146,0.131677,0.000003,nan,0,0,5137,40
1,4096,1,21548,45665,2.119222,0.175189,0.000008,nan,0,0,5132,52


In [29]:
compare([do_render_code("linked_list/linked_list.s", show=["depth_of_array:",".LFE40:"]),do_render_code("linked_list/linked_list.s", show=["depth_of_list",".LFE39:"])])

In [30]:
! make -C linked_list clean ; make -C linked_list
! echo "size,list,IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branches,branch_misses" > stats_small.csv
! echo -n "1024,0," >> stats_small.csv
! linked_list/linked_list 8192 0 stats_small.csv
! echo -n "1024,1," >> stats_small.csv
! linked_list/linked_list 8192 1 stats_small.csv

make: Entering directory '/nfshome/htseng/courses/CS203/demo/OoO_programming/linked_list'
rm -f linked_list linked_list.s
make: Leaving directory '/nfshome/htseng/courses/CS203/demo/OoO_programming/linked_list'
make: Entering directory '/nfshome/htseng/courses/CS203/demo/OoO_programming/linked_list'
cc -O3 -Wno-implicit-function-declaration -Wno-format-zero-length -DHAVE_LINUX_PERF_EVENT_H perfstats.c  linked_list.c -o linked_list
cc -S -O3 -Wno-implicit-function-declaration -Wno-format-zero-length -DHAVE_LINUX_PERF_EVENT_H linked_list.c 
make: Leaving directory '/nfshome/htseng/courses/CS203/demo/OoO_programming/linked_list'
depth: 8192
depth: 8192


In [31]:
display_df_mono(render_csv("stats_small.csv"))

,size,list,IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branches,branch_misses
0,1024,0,54376,41802,0.768758,0.167456,0.000007,nan,0,0,9231,40
1,1024,1,37932,113302,2.986977,0.176519,0.000020,nan,0,0,9231,48


## Why is the CPI longer than what we expected? 

Any thought?

# Code example: popcount

Given a 64-bit integer number, find the number of 1s in its binary representation.
Example 1:
Input: 59487
Output: 9

Explanation: 9487’s binary representation is 0b10110010100001111

In [38]:
render_code("popcounts/popcount.cpp", show=["#else //A", "#endif"])

// popcounts/popcount.cpp:80-90 (11 lines)
#else //A
inline int popcount(uint64_t x){
	int c=0;
	while(x) //while any bits are set not sure how you are getting the value
    {
       c += x & 1;
       x = x >> 1;
    }
    return c;
}
#endif

In [39]:
render_code("popcounts/popcount.cpp", show=["#ifdef B", "#else"])

// popcounts/popcount.cpp:25-41 (17 lines)
#ifdef B
inline int popcount(uint64_t x) {
     int c = 0;
	 while(x) //while any bits are set not sure how you are getting the value
     {
       c += x & 1;
       x = x >> 1;
       c += x & 1;
       x = x >> 1;
       c += x & 1;
       x = x >> 1;
       c += x & 1;
       x = x >> 1;
     }
     return c;
}
#else

In [40]:
render_code("popcounts/popcount.cpp", show=["#ifdef C", "#else"])

// popcounts/popcount.cpp:13-24 (12 lines)
#ifdef C
inline int popcount(uint64_t x) {
     int c = 0;
     int table[16] = {0, 1, 1, 2, 1, 2, 2, 3, 1, 2, 2, 3, 2, 3, 3, 4};
     while(x) {
	       c += table[(x & 0xF)];
	       x = x >> 4;
     }
     return c;
}

#else

In [41]:
render_code("popcounts/popcount.cpp", show=["#ifdef D", "#else"])

// popcounts/popcount.cpp:42-52 (11 lines)
#ifdef D
inline int popcount(uint64_t x) {
     int c = 0;
     int table[16] = {0, 1, 1, 2, 1, 2, 2, 3, 1, 2, 2, 3, 2, 3, 3, 4};
     for (uint64_t i = 0; i < 16; i++) {
	       c += table[(x & 0xF)];
	       x = x >> 4;
     }
     return c;
}
#else

In [42]:
render_code("popcounts/popcount.cpp", show=["#ifdef E", "#else"])

// popcounts/popcount.cpp:53-80 (28 lines)
#ifdef E
inline int popcount(uint64_t x) {
     int c = 0;
     for (uint64_t i = 0; i < 16; i++) {
         switch((x & 0xF))
         {
             case 1: c+=1; break;
             case 2: c+=1; break;
             case 3: c+=2; break;
             case 4: c+=1; break;
             case 5: c+=2; break;
             case 6: c+=2; break;
             case 7: c+=3; break;
             case 8: c+=1; break;
             case 9: c+=2; break;
             case 10: c+=2; break;
             case 11: c+=3; break;
             case 12: c+=2; break;
             case 13: c+=3; break;
             case 14: c+=3; break;
             case 15: c+=4; break;
             default: break;
         }
         x = x >> 4;
     }
     return c;
}
#else //A

## Who has the best performance?

In [ ]:
render_code("popcounts/popcount.cpp", show="main")

In [ ]:
! make -C popcounts clean; make -C popcounts
! echo "Version A"; time ./popcounts/popcount_A
! echo "Version B";time ./popcounts/popcount_B
! echo "Version C";time ./popcounts/popcount_C
! echo "Version D";time ./popcounts/popcount_D
! echo "Version E";time ./popcounts/popcount_E

make: Entering directory '/nfshome/htseng/courses/CS203/demo/OoO_programming/popcounts'
rm -f popcount_A popcount_B popcount_C popcount_D popcount_SSE42 popcount_E
make: Leaving directory '/nfshome/htseng/courses/CS203/demo/OoO_programming/popcounts'
make: Entering directory '/nfshome/htseng/courses/CS203/demo/OoO_programming/popcounts'
g++ -O3 popcount.cpp -o popcount_A
g++ -S -O3 popcount.cpp -o A.s
g++ -O3 -DB popcount.cpp -o popcount_B
g++ -S -O3 -DB popcount.cpp -o B.s
g++ -O3 -DC popcount.cpp -o popcount_C
g++ -S -O3 -DC popcount.cpp -o C.s
g++ -O3 -DD popcount.cpp -o popcount_D
g++ -S -O3 -DD popcount.cpp -o D.s
g++ -O3 -m64 -msse4.2 -DSSE42 popcount.cpp -o popcount_SSE42
g++ -O3 -DE popcount.cpp -o popcount_E
g++ -S -O3 -DE popcount.cpp -o E.s
make: Leaving directory '/nfshome/htseng/courses/CS203/demo/OoO_programming/popcounts'
Version A
Result: 32000010811

real	0m22.364s
user	0m22.358s
sys	0m0.000s
Version B
Result: 32000010811

real	0m12.335s
user	0m12.332s
sys	0m0.000s
Ver

In [51]:
! echo "Version SSE";time ./popcounts/popcount_SSE42

Version SSE
Result: 32000010811

real	0m1.608s
user	0m1.592s
sys	0m0.000s


In [45]:
! perf stat ./popcounts/popcount_B

Result: 32000010811

 Performance counter stats for './popcounts/popcount_B':

         12,293.21 msec task-clock                #    0.999 CPUs utilized          
                45      context-switches          #    3.661 /sec                   
                 1      cpu-migrations            #    0.081 /sec                   
               124      page-faults               #   10.087 /sec                   
    63,671,869,739      cycles                    #    5.179 GHz                    
   287,893,899,490      instructions              #    4.52  insn per cycle         
    17,938,210,884      branches                  #    1.459 G/sec                  
        63,538,773      branch-misses             #    0.35% of all branches        

      12.302937765 seconds time elapsed

      12.293442000 seconds user
       0.000000000 seconds sys




In [44]:
! perf stat ./popcounts/popcount_C

Result: 32000010811

 Performance counter stats for './popcounts/popcount_C':

          5,020.51 msec task-clock                #    0.996 CPUs utilized          
                18      context-switches          #    3.585 /sec                   
                 1      cpu-migrations            #    0.199 /sec                   
               125      page-faults               #   24.898 /sec                   
    25,978,497,649      cycles                    #    5.174 GHz                    
   102,680,019,492      instructions              #    3.95  insn per cycle         
    17,935,701,642      branches                  #    3.572 G/sec                  
        67,096,171      branch-misses             #    0.37% of all branches        

       5.040950415 seconds time elapsed

       5.021209000 seconds user
       0.000000000 seconds sys




In [47]:
! perf stat ./popcounts/popcount_D

Result: 32000010811

 Performance counter stats for './popcounts/popcount_D':

          3,738.54 msec task-clock                #    1.000 CPUs utilized          
                13      context-switches          #    3.477 /sec                   
                 1      cpu-migrations            #    0.267 /sec                   
               124      page-faults               #   33.168 /sec                   
    19,354,371,200      cycles                    #    5.177 GHz                    
    80,011,090,228      instructions              #    4.13  insn per cycle         
     1,001,959,854      branches                  #  268.008 M/sec                  
            23,287      branch-misses             #    0.00% of all branches        

       3.740188226 seconds time elapsed

       3.738764000 seconds user
       0.000000000 seconds sys




In [49]:
! perf stat ./popcounts/popcount_E

Result: 32000010811

 Performance counter stats for './popcounts/popcount_E':

         54,665.26 msec task-clock                #    1.000 CPUs utilized          
               230      context-switches          #    4.207 /sec                   
                19      cpu-migrations            #    0.348 /sec                   
               124      page-faults               #    2.268 /sec                   
   282,193,484,772      cycles                    #    5.162 GHz                    
   173,110,589,099      instructions              #    0.61  insn per cycle         
    44,019,686,608      branches                  #  805.259 M/sec                  
     8,173,883,973      branch-misses             #   18.57% of all branches        

      54.668226560 seconds time elapsed

      54.665983000 seconds user
       0.000000000 seconds sys




In [50]:
! perf stat ./popcounts/popcount_A

Result: 32000010811

 Performance counter stats for './popcounts/popcount_A':

         22,538.18 msec task-clock                #    1.000 CPUs utilized          
               118      context-switches          #    5.236 /sec                   
                18      cpu-migrations            #    0.799 /sec                   
               126      page-faults               #    5.591 /sec                   
   116,471,516,206      cycles                    #    5.168 GHz                    
   332,048,770,348      instructions              #    2.85  insn per cycle         
    65,008,748,675      branches                  #    2.884 G/sec                  
       740,247,718      branch-misses             #    1.14% of all branches        

      22.540992522 seconds time elapsed

      22.538426000 seconds user
       0.000000000 seconds sys




In [55]:
render_code("popcounts/popcount.cpp", show=["#ifdef SSE42", "#else"])

// popcounts/popcount.cpp:5-11 (7 lines)
#ifdef SSE42
#include <smmintrin.h>
inline int popcount(uint64_t x) {
     int c = _mm_popcnt_u64(x);
     return c;
}
#else

In [53]:
! perf stat ./popcounts/popcount_SSE42

Result: 32000010811

 Performance counter stats for './popcounts/popcount_SSE42':

          1,575.33 msec task-clock                #    1.000 CPUs utilized          
                11      context-switches          #    6.983 /sec                   
                 4      cpu-migrations            #    2.539 /sec                   
               123      page-faults               #   78.079 /sec                   
     8,135,453,507      cycles                    #    5.164 GHz                    
    22,006,768,381      instructions              #    2.71  insn per cycle         
     1,001,183,306      branches                  #  635.538 M/sec                  
            21,461      branch-misses             #    0.00% of all branches        

       1.575972179 seconds time elapsed

       1.575859000 seconds user
       0.000000000 seconds sys




## How to implement a branch?


In [ ]:
compare([do_render_code("branch/branch.c",show=["#ifdef BITWISE","#else"]),do_render_code("branch/branch.c",show=["#else // B","#endif"]),do_render_code("branch/branch.c",show=["#ifdef NO_BRANCH","#else"]),do_render_code("branch/branch.c",show=["#ifdef COND","#else"])])

In [ ]:
! make -C branch clean; make -C branch
! perf stat branch/branch_A 1024 16777216
! perf stat branch/branch_B 1024 16777216
! perf stat branch/branch_C 1024 16777216
! perf stat branch/branch_D 1024 16777216

## ILP of your program

In [ ]:
! perf stat ./popcounts/popcount_D